In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetSpec, ExecutionConfiguration, DerivaML, Workflow
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'dev.eye-ai.org'
host = 'www.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

# Configuration

In [ ]:
# RID of source dataset, if any.
source_dataset = '2-C9PR' # '2-7P5P'#'2-AGAW'

workflow_instance = EA.create_workflow(
    name="Multimodal template",
    workflow_type="Test Workflow",
)

config = ExecutionConfiguration(
    datasets=[DatasetSpec(rid=source_dataset, version=EA.dataset_version(source_dataset), materialize=False)],
    assets=[],
    workflow= workflow_instance, # dev'5-SG9W'
    description="Template instance of a feature creation workflow")

# Initialize execution
execution = EA.create_execution(config)


In [ ]:
print(execution)

# Create DatasetBag

In [ ]:
ds_bag = execution.datasets[0]

# Methods for multimodal data

### Extract modality

In [ ]:
modalities = EA.extract_modality(ds_bag)

In [ ]:
modalities['HVF']

In [ ]:
len(ds_bag.get_table_as_dataframe('Subject')) # 1180
# ds_bag.get_table_as_dataframe('OCR_HVF')

### Retrive multimodal wide table

In [ ]:
wide = EA.multimodal_wide(ds_bag)

In [ ]:
print(len(wide))
wide.columns

In [ ]:
wide[wide['MD'].isna()]

In [ ]:
wide.to_csv('new_wide.csv')

# Upload results

In [ ]:
# crete asset path

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)